## Scraping Mango product images

In [1]:
import numpy as np
import time
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import urllib
import cv2
from selenium import webdriver
!pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

### 1. Get link to each product page

In [2]:
url = "https://shop.mango.com/us/women/bags_c18162733"
# I used Firefox; you can use whichever browser you like.
browser = webdriver.Chrome(ChromeDriverManager().install())

# Tell Selenium to get the URL you're interested in.
browser.get(url)

# Selenium script to grab all product info on page -> wait 3 seconds for the next batch of data to load, 
# then continue scrolling. It will continue to do this until the page stops loading new data.
all_ = []
hrefs = []
desc = []
prices = []

lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    elements = (browser.find_elements_by_class_name(name='aZ-72'))
    all_.extend(elements)
    hrefs.extend([e.get_attribute('href') for e in elements])
    elements2 = (browser.find_elements_by_class_name(name='_1P8s4'))
    desc.extend([e.text for e in elements2])
    elements3 = (browser.find_elements_by_class_name(name= 'B16Le'))
    prices.extend([e.text for e in elements3])
    time.sleep(3)
    
    if lastCount==lenOfPage:
        match=True
    
# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source
browser.close()


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver


In [3]:
print(len(hrefs)) #309 #https://shop.mango.com/us/women/bags-shoulder-bags/croc-effect-bag_43070705.html?c=30&n=1&s=accesorios_she.accesorio;40,340,440'
# hrefs = hrefs[157:]
# print(len(linhks2))

152


In [4]:
len(prices)

152

In [5]:
len(desc)

152

In [6]:
all_data = []

for i in range(len(hrefs)):
    all_data.append({'href': hrefs[i],
                    'price': prices[i],
                    'description': desc[i],
                    'brand': 'Mango',
                    'num': i})

In [7]:
df_data = pd.DataFrame(all_data)

In [8]:
df_data

,brand,description,href,num,price
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,0,$49.99
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,1,$79.99
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,2,$69.99
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,3,$59.99
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,4,$25.99
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,5,$99.99
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,6,$59.99
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,7,$99.99
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,8,$39.99
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,9,$59.99


In [9]:
df_data.drop_duplicates(subset='href', inplace=True)
df_data

,brand,description,href,num,price
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,0,$49.99
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,1,$79.99
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,2,$69.99
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,3,$59.99
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,4,$25.99
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,5,$99.99
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,6,$59.99
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,7,$99.99
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,8,$39.99
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,9,$59.99


In [21]:
df_data.drop(columns='num', inplace = True)

In [23]:
df_data.price = df_data.price.apply(lambda x: x.replace('$', ''))
df_data.price = df_data.price.apply(lambda x: float(x))
df_data

,brand,description,href,price
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99


In [28]:
df_data.drop(index=[23,29,149], axis = 0, inplace=True)

In [29]:
df_data

,brand,description,href,price
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99


In [31]:
num = list(range(len(df_data)))
num

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148]

In [33]:
df_data = pd.concat([df_data, pd.Series(num)], axis=1)

In [41]:
df_data

,brand,description,href,price,0,img-file
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99,0.0,mango_1.0.png
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99,1.0,mango_2.0.png
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99,2.0,mango_3.0.png
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,3.0,mango_4.0.png
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99,4.0,mango_5.0.png
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99,5.0,mango_6.0.png
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,6.0,mango_7.0.png
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99,7.0,mango_8.0.png
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99,8.0,mango_9.0.png
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99,9.0,mango_10.0.png


In [38]:
df_data['img-file'] = df_data[0].apply(lambda x: 'mango_'+str(int(x)+1)+'.png')
df_data

ValueError: cannot convert float NaN to integer

In [42]:
df_data['img-file'] = df_data['img-file'].apply(lambda x: x.replace('.0', ''))

In [43]:
df_data

,brand,description,href,price,0,img-file
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99,0.0,mango_1.png
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99,1.0,mango_2.png
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99,2.0,mango_3.png
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,3.0,mango_4.png
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99,4.0,mango_5.png
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99,5.0,mango_6.png
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,6.0,mango_7.png
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99,7.0,mango_8.png
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99,8.0,mango_9.png
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99,9.0,mango_10.png


In [44]:
df = pd.read_csv('mango_data_updated.csv')

In [45]:
df

,Unnamed: 0,brand,description,href,price
0,0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99
1,1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99
2,2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99
3,3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
4,4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99
5,5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99
6,6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99
7,7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99
8,8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99
9,9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99


In [46]:
num = list(range(len(df)))

In [48]:
df= pd.concat([df, pd.Series(num)], axis = 1)
df

,Unnamed: 0,brand,description,href,price,0
0,0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99,0
1,1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99,1
2,2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99,2
3,3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,3
4,4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99,4
5,5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99,5
6,6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,6
7,7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99,7
8,8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99,8
9,9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99,9


In [49]:
df['img-file'] = df[0].apply(lambda x: 'mango_'+str(x+1)+'.png')
df

,Unnamed: 0,brand,description,href,price,0,img-file
0,0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99,0,mango_1.png
1,1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99,1,mango_2.png
2,2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99,2,mango_3.png
3,3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,3,mango_4.png
4,4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99,4,mango_5.png
5,5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99,5,mango_6.png
6,6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,6,mango_7.png
7,7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99,7,mango_8.png
8,8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99,8,mango_9.png
9,9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99,9,mango_10.png


In [51]:
df.drop(columns=['Unnamed: 0', 0], inplace=True)

In [52]:
df

,brand,description,href,price,img-file
0,Mango,Croc-effect bag,https://shop.mango.com/us/women/bags-shoulder-...,49.99,mango_1.png
1,Mango,Round raffia bag,https://shop.mango.com/us/women/bags-raffia-an...,79.99,mango_2.png
2,Mango,Metallic-effect shopper bag,https://shop.mango.com/us/women/bags-shoppers/...,69.99,mango_3.png
3,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,mango_4.png
4,Mango,Handmade net bag,https://shop.mango.com/us/women/bags-shoppers/...,25.99,mango_5.png
5,Mango,Bamboo shells bag,https://shop.mango.com/us/women/bags-raffia-an...,99.99,mango_6.png
6,Mango,Leather mini bag,https://shop.mango.com/us/women/bags-small/lea...,59.99,mango_7.png
7,Mango,Bow leather bag,https://shop.mango.com/us/women/bags-handbags/...,99.99,mango_8.png
8,Mango,Anim print crossbody belt bag,https://shop.mango.com/us/women/bags-small/ani...,39.99,mango_9.png
9,Mango,Croc-effect mini bag,https://shop.mango.com/us/women/bags-small/cro...,59.99,mango_10.png


In [53]:
df.to_csv('mango_data_updated.csv')

In [30]:
df_data.to_csv('mango_data_updated.csv')

In [24]:
df_data.to_csv('mango_data.csv')

### 2. Loop through each product href to get image link

In [10]:
df_data.href[0]

'https://shop.mango.com/us/women/bags-shoulder-bags/croc-effect-bag_43070705.html?c=30&n=1&s=accesorios_she.accesorio;40,340,440'

In [14]:
links3 = []
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(df_data.href[0])
elem = driver.find_element_by_class_name('image-1')
print(elem.get_attribute('src'))
# links3.append(elem.get_attribute['src'])
driver.close()
# links3



Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43070705_30-99999999_01.jpg?ts=1554972028823&imwidth=480&imdensity=2


In [15]:
links = []
links_ = []

for i, href in enumerate(df_data.href):
    try:
        time.sleep(2)
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(href)
        elem1 = driver.find_element_by_class_name('image-1')
        links_.append(elem1.get_attribute('src'))
        elem = driver.find_element_by_class_name('image-2')
        links_.append(elem.get_attribute('src'))
        driver.close()
        print(i)
    except:
        links.append('no such src')


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
0

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
1

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
2

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
3

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
4

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
5

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
6

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
7


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
65

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
66

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
67

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
68

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
69

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
70

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
71

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedri

Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
129

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
130

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
131

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
132

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
133

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
134

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
135

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/gracejeong/.wdm/chromedriver/2.46/mac64/chromedriver
136

Checking for mac64 chromedrive

In [16]:
for l_ in links_:
    print(l_)

['https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43070705_30-99999999_01.jpg?ts=1554972028823&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070705_30_B.jpg?ts=1554972028823&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43035019_08-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035019_08_B.jpg?ts=1550136842788&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43005025_PL-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005025_PL_B.jpg?ts=1549294423965&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43035022_43-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035022_43_B.jpg?ts=1550136842788&imwidth=480&imdensity=2',
 'https://st.mngbcn.

In [27]:
for l_ in links_:
    print(l_)

https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43070705_30-99999999_01.jpg?ts=1554972028823&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070705_30_B.jpg?ts=1554972028823&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43035019_08-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035019_08_B.jpg?ts=1550136842788&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43005025_PL-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005025_PL_B.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43035022_43-99999999_01.jpg?ts=1554921220447&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035022_43_B.jpg?ts=1550136842788&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/4

https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43023763_99_B.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43023763_99.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43005026_PL-99999999_01.jpg?ts=1550846029131&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005026_PL_B.jpg?ts=1549622471828&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005027_99_B.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005027_99.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43005027_43-99999999_01.jpg?ts=1551954446192&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005027_43_B.jpg?ts=1549294423965&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43050612_99_B.jpg?ts=1538652813275&imwi

https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43090934_99_B.jpg?ts=1544006027145&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43090934_99.jpg?ts=1544006027145&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43095786_30-99999999_01.jpg?ts=1551951207887&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43095786_30_B.jpg?ts=1551083590481&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43070788_99-99999999_01.jpg?ts=1545053621734&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070788_99_B.jpg?ts=1541158394159&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43070836_07-99999999_01.jpg?ts=1550241232956&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070836_07_B.jpg?ts=1550489621996&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43050819_99_B.jpg?ts

https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070935_99.jpg?ts=1544006027145&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43083729_99-99999999_01.jpg?ts=1550052677425&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43083729_99_B.jpg?ts=1545237239285&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43040937_99_B.jpg?ts=1544006027145&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43040937_99.jpg?ts=1544006027145&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43083729_CU_B.jpg?ts=1545237239285&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43083729_CU.jpg?ts=1545237239285&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43045797_CU-99999999_01.jpg?ts=1550241232956&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43045797_CU_B.jpg?ts=1549373639810&imwidt

https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43073030_52_B.jpg?ts=1547544832276&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43073030_52.jpg?ts=1547541205099&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43020828_43_B.jpg?ts=1544024827713&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43020828_43.jpg?ts=1544024827713&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43043717_05_B.jpg?ts=1541522241796&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43043717_05.jpg?ts=1541522241796&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43023006_99_B.jpg?ts=1541522241796&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43023006_99.jpg?ts=1541522241796&imwidth=480&imdensity=2
https://st.mngbcn.com/rcs/pics/static/T4/fotos/outfit/S20/43045015_99-99999999_01.jpg?ts=1554392000010&imwidth=480&imdensity=2
h

In [17]:
len(links_)

299

In [25]:
links_[-4:]

['https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070747_30_B.jpg?ts=1539858797909&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070747_30.jpg?ts=1539855212942&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035791_08_B.jpg?ts=1547544832276&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035791_08.jpg?ts=1547541205099&imwidth=480&imdensity=2']

In [12]:
links

['https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43070705_30_B.jpg?ts=1554972028823&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035019_08_B.jpg?ts=1550136842788&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43005025_PL_B.jpg?ts=1549294423965&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035022_43_B.jpg?ts=1550136842788&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43033720_52.jpg?ts=1544006027145&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43085787_08_B.jpg?ts=1552577975602&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43035022_23_B.jpg?ts=1549290906437&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43045810_99_B.jpg?ts=1549632809999&imwidth=480&imdensity=2',
 'https://st.mngbcn.com/rcs/pics/static/T4/fotos/S20/43080591_70.jpg?ts=1545917588393&imwi

In [13]:
len(links)

152

### 3. Loop through each image link and save locally

In [19]:
for i, url in enumerate(links_):
    time.sleep(1)
    try:
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        binary_str = response.read()
        byte_array = bytearray(binary_str)
        numpy_array = np.asarray(byte_array, dtype='int8')
        image = cv2.imdecode(numpy_array, cv2.IMREAD_UNCHANGED)
        cv2.imwrite("Mango2/"+"mango_{}".format(i+1)+".png", image)
        print("Saved "+"mango_{}".format(i+1)+".png")
    except Exception as e:
        print(str(e))

print("Done")

Saved mango_1.png
Saved mango_2.png
Saved mango_3.png
Saved mango_4.png
Saved mango_5.png
Saved mango_6.png
Saved mango_7.png
Saved mango_8.png
Saved mango_9.png
Saved mango_10.png
Saved mango_11.png
Saved mango_12.png
Saved mango_13.png
Saved mango_14.png
Saved mango_15.png
Saved mango_16.png
Saved mango_17.png
Saved mango_18.png
Saved mango_19.png
Saved mango_20.png
Saved mango_21.png
Saved mango_22.png
Saved mango_23.png
Saved mango_24.png
Saved mango_25.png
Saved mango_26.png
Saved mango_27.png
Saved mango_28.png
Saved mango_29.png
Saved mango_30.png
Saved mango_31.png
Saved mango_32.png
Saved mango_33.png
Saved mango_34.png
Saved mango_35.png
Saved mango_36.png
Saved mango_37.png
Saved mango_38.png
Saved mango_39.png
Saved mango_40.png
Saved mango_41.png
Saved mango_42.png
Saved mango_43.png
Saved mango_44.png
Saved mango_45.png
Saved mango_46.png
Saved mango_47.png
Saved mango_48.png
Saved mango_49.png
Saved mango_50.png
Saved mango_51.png
Saved mango_52.png
Saved mango_53.png
Sa

In [26]:
299/2

149.5